Les Misérables

## llm 설정

In [1]:
from langchain_community.chat_models import ChatOllama
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv,find_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatMessagePromptTemplate,ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate,MessagesPlaceholder
from langchain import LLMChain
load_dotenv()

True

In [2]:
llm_model = ChatOllama(model='llama3.1:latest')
#response_llm = llm_model.invoke("Hello, how can I help you today?")
#print(response_llm.content)


In [3]:
gpt_model = ChatOpenAI(model_name="gpt-4")
#response_gpt = gpt_model.invoke("Hello, how can I help you today?")
#print(response_gpt.content)

c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


## 1. llm 모델 작성

### 1-1. 프롬프트 작성

In [4]:
system_message = """
    너는 책을 읽어주는 어시스턴트야. 아래 지침에 따라 답변을 해줘야 해.
    모든 출력은 한국어로 해야 해.
    
    1. 사용자가 txt 파일 내용을 제공하면, 해당 내용을 문단별로 출력해야 해.
    2. 사용자가 '다음'이라는 질문({question})을 하면, 다음 문단을 원문대로 출력해야 해.
    3. txt 파일 내용을 처음 받으면, {context_character}의 말투를 참고하여 등장인물처럼 대화해야 해.
    4. 문단 중에 " ", ' ' 와 같은 대화문이 나오면, 대화문 뒤에 : {emotion}을 함께 출력해야 해.
    5. 사용자가 책의 내용에 대해 질문({question})을 하면, 등장인물의 말투로 {context_books}를 참고하여 답변해야 해.

"""

In [5]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        MessagesPlaceholder(variable_name="history"), # 이전 대화 내용 저장용
        ("human", "{input}")
    ]
)
output_parser= StrOutputParser()#글자만 가져온다


### 1-2 대화 내용을 기억하는 llm 만들기

In [6]:
# 대화 메모리 생성 (대화 히스토리를 관리)
conversation_memory = ConversationBufferMemory()


# LLMChain 생성: 프롬프트와 LLM을 연결
llm_chain = LLMChain(
    llm=gpt_model,
    prompt=chat_prompt,
)

# 사용자 질문 처리 및 답변 생성 함수
def handle_user_input(user_input, history=[], context_books="책 내용 예시", context_character="장발장", emotion="중립", question="질문 예시",txt_path = None,paragraph_index=0):
    # 히스토리 업데이트
    history.append({"role": "user", "content": user_input})
    
    ### 파일 경로 수정하기
    if txt_path is not None :
        with open(txt_path, 'r', encoding='utf-8') as file:
            file_content = file.read()

        # 문단별로 나누기
        paragraphs = file_content.split('\n\n')  # 빈 줄을 기준으로 문단 분리
        print(len(paragraphs))
        print(paragraph_index)
        if paragraph_index < len(paragraphs):
            user_input = paragraphs[paragraph_index]  # 해당 인덱스의 문단을 user_input으로 설정
            history.append({"role": "user", "content": user_input})
        else:
            user_input = "모든 문단이 끝났습니다."
    # 모델 실행 및 답변 생성
    response = llm_chain.run({
        "history": history,
        "input": user_input,
        "context_books": context_books,
        "context_character": context_character,
        "emotion": emotion,
        "question": question
    })
    
    # 히스토리에 응답 추가
    history.append({"role": "assistant", "content": response})
    
    return response, history

# 예시 실행
user_input = "첫 문단을 읽어줘"
history = []
#paragraph_index = 3  # 첫 번째 문단부터 시작
response, history = handle_user_input(user_input, history,txt_path="books/LES MISERABLES_ch1.txt")
print(response)


c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


19
0
"LES MISÉRABLES
서문
법과 풍습에 의해, 사회가 정한 저주의 선고가 존재하고, 인위적으로 지구의 문명 속에서 지옥을 창조하며, 인간의 운명 요소를 신의 운명에 더하고 있는 한; 세기의 세 대문제인 빈곤을 통한 인간의 저하, 배고픔을 통한 여성의 부패, 빛의 부족으로 인한 아이들의 장애가 해결되지 않는 한; 세상 어디에서든 사회적 숨막힘 현상이 가능한 한; 다시 말해, 더 넓은 의미에서는, 지구 상에 무지와 빈곤이 존재하는 한, 레미제라블 같은 성격의 책은 반드시 유용하지 않을 수 없다."


In [7]:
# 테스트용 챗봇
paragraph_index = 0
txt_path = 'books/LES MISERABLES_ch1.txt'
while True:
    user_input = input("당신: ")
    if user_input.lower() in ["exit", "quit", "종료"]:
        print("대화를 종료합니다.")
        break
    response, history = handle_user_input(user_input, history,txt_path=txt_path,paragraph_index = paragraph_index)
    paragraph_index +=1
    print(f"어시스턴트: {response}")

19
0
어시스턴트: "LES MISÉRABLES
서문
법과 풍습에 의해, 사회가 정한 저주의 선고가 존재하고, 인위적으로 지구의 문명 속에서 지옥을 창조하며, 인간의 운명 요소를 신의 운명에 더하고 있는 한; 세기의 세 대문제인 빈곤을 통한 인간의 저하, 배고픔을 통한 여성의 부패, 빛의 부족으로 인한 아이들의 장애가 해결되지 않는 한; 세상 어디에서든 사회적 숨막힘 현상이 가능한 한; 다시 말해, 더 넓은 의미에서는, 지구 상에 무지와 빈곤이 존재하는 한, 레미제라블 같은 성격의 책은 반드시 유용하지 않을 수 없다."
19
1
어시스턴트: "HAUTEVILLE HOUSE, 1862."
19
2
어시스턴트: "제1권
팡틴
제1부 - 정의로운 사람
제1장 - M. 미리엘
1815년, M. 샤를-프랑수아-비엔베뉴 미리엘은 D——의 주교였다. 그는 대략 75세의 노인이었고, 1806년부터 D——의 주교로 있었다."
19
3
어시스턴트: "이 세부 정보가 우리가 이야기하려는 실제 내용과는 전혀 관련이 없음에도 불구하고, 모든 점에서의 정확성을 위해, 그가 교구에 도착한 순간부터 그에 대해 유행하던 여러 소문과 언급을 여기에 언급하는 것은 불필요하지 않을 것이다. 참이든 거짓이든, 사람들에게 말하는 것은 그들의 삶에서, 그리고 무엇보다 그들의 운명에서, 그들이 하는 것만큼 중요한 위치를 차지한다. M. 미리엘은 에이크스 의회의 한 상담원의 아들이었다; 따라서 그는 바의 귀족에 속했다. 그의 아버지는 그를 자신의 직위의 후계자로 만들기 위해, 국회 가족들 사이에서 상당히 널리 퍼져 있는 관습에 따라, 그를 아주 어린 나이, 18살이나 20살에 결혼시켰다고 전해진다. 그러나 이 결혼에도 불구하고, 샤를 미리엘이 많은 이야기를 만들어냈다고 전해진다. 그는 잘 생겼지만 키가 조금 작고, 우아하고, 매력적이며, 똑똑했다; 그의 인생의 첫 부분 전체가 세상과 용감함에 헌신되었다."
19
4


KeyboardInterrupt: 

In [8]:
# txt 파일 내용을 읽고 챗봇에 전달 -gradio 용 잘 안됨
def handle_txt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        file_content = file.read()

    # 파일 내용을 챗봇에게 전달하고 첫 문단을 출력
    response = handle_user_input(file_content)
    print(f"어시스턴트: {response}")
    return file_content

### 2. 텍스트 벡터화 임베딩

### 챗봇 ui

### 2-1 백터화 준비

In [14]:
from langchain_openai import OpenAIEmbeddings
import os
from langchain.docstore.document import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
import shutil
import io
TXT_PATH = "txt_files"
OPENAI_DB_PATH = "./openai_db"
GRADIO_FILE_PATH = ""
emb = OpenAIEmbeddings()
docs=[]
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, # 각 조각의 최대 문자 수를 1000자로 설정합니다.
    chunk_overlap = 100  # 각 조각이 서로 겹치는 부분을 100자로 설정하여 분할된 텍스트 간의 연결성을 유지합니다.
    )

### 2-2 파일 업로드 처리

In [15]:
save_dir = "save_txt"
def save_uploaded_file(file):

    os.makedirs(save_dir, exist_ok=True)
    file_path = os.path.join(save_dir, os.path.basename(file.name))
    if file is not None:
        # file 은 임시 저장 결로를 나타내는 듯
        shutil.copy(file, file_path)
    print("저장되었음")
    return f"파일이 저장되었습니다"

In [17]:
# 파일 벡터화 임베딩
def text_vectorize(file):
    file_path = os.path.join(save_dir, os.path.basename(file.name))

    if not os.path.exists(file_path):
            print(f"File not found at: {file_path}")
            return None

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            doc = Document(page_content=content)
            docs.append(doc)
            docs_spliter = text_splitter.split_documents(docs)
            openai_vectorstore = Chroma.from_documents(
                docs_spliter,
                emb,
                persist_directory=OPENAI_DB_PATH
            )
    except Exception as e:
        print(f"Error reading file: {e}")
        return None
    

    # 데이터가 잘 저장되었는지 확인하는 부분
    try:
        doc_count = openai_vectorstore._collection.count()
        print(f"벡터 저장소에 저장된 문서 수: {doc_count}")
        
        # 예시로 첫 번째 문서 임베딩을 확인해보기
        if doc_count > 0:
            sample_result = openai_vectorstore.similarity_search(docs_spliter[0].page_content)
            print(f"첫 번째 문서와 유사한 문서 검색 결과: {sample_result}")
    except Exception as e:
        print(f"Error verifying vector store: {e}")
        return None

    return openai_vectorstore

In [18]:
# 파일 문단별 분리

In [19]:
# 파일 업로드 처리 함수
# 이거 수정해서 벡터화 및 파일 db 생성해서 순서대로 부르게 하기
# gpt 보라
def chatbot_send_file(chat_history, file):

    save_uploaded_file(file)
    text_vectorize(file)

    chat_history.append(["사용자가 업로드한 파일입니다.",f"{file.name}"])
    return chat_history

In [20]:
# 이벤트 처리 함수
def chatbot_send_message(chat_history, text):
    ai_text = llm_chain.invoke(text)  # 여기에 챗봇 처리 로직 추가
    chat_history.append([text, ai_text])
    return chat_history, ""

In [21]:
import gradio as gr


with gr.Blocks() as app:
    with gr.Tab("음성 인식봇"):
        with gr.Column():
            gr.Markdown(
                value="""
<center><h1> 음성 인식 봇 </h1></center>
<center> AI 인공지능 봇입니다. 음성으로 묻거나 문서 요약, 일정 관리 등을 할 수 있습니다.</center>
"""
            )
            cb_chatbot = gr.Chatbot(
                value=[[None, "안녕하세요 AI 챗봇입니다. 무엇이든 질문하세요"],
                       ['나는 지금 배가 고파요', '배가 고프시군요 메뉴를 추천해드릴까요?']
                       ],
                label="구미챗봇",
            )
        with gr.Row():
            cb_textbox = gr.Textbox(
                lines=3,
                placeholder="입력창입니다.",
                container=False,
                scale=7
            ) # 채팅 입력

            gr.Audio(
                format="mp3",
                scale=1,
                min_width=200,
                label="음성을 입력해주세요"
            ) # 음성 입력

            cb_send_btn = gr.Button(
                value="보내기",
                visible="primary",
                scale=1
            ) # 실행 버튼
            
            file_upload = gr.File(
                label="txt 파일을 업로드해주세요",
                type="filepath"
            ) # 파일 업로드

        with gr.Row():
            gr.Button(
                value="되돌리기"
            )
            gr.Button(
                value="초기화"
            )
        
        # 텍스트 입력 처리
        cb_send_btn.click(
            fn=chatbot_send_message,
            inputs=[cb_chatbot, cb_textbox],
            outputs=[cb_chatbot, cb_textbox]
        )
        
        # 파일 업로드 처리
        file_upload.change(
            fn= chatbot_send_file,
            inputs = [cb_chatbot,file_upload],#입력값 전달될 것 
            outputs=cb_chatbot#출력값 들어갈 곳
        )

    with gr.Tab("문서 요약봇"):
        pass
    with gr.Tab("일정 관리봇"):
        pass

app.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


저장되었음
벡터 저장소에 저장된 문서 수: 194
첫 번째 문서와 유사한 문서 검색 결과: [Document(page_content='CHAPTER I—M. MYRIEL\nIn 1815, M. Charles-François-Bienvenu Myriel was Bishop of D—— He was an old man of about seventy-five years of age; he had occupied the see of D—— since 1806.'), Document(page_content='In 1804, M. Myriel was the Curé of B—— [Brignolles]. He was already advanced in years, and lived in a very retired manner.\n\nAbout the epoch of the coronation, some petty affair connected with his curacy—just what, is not precisely known—took him to Paris. Among other powerful persons to whom he went to solicit aid for his parishioners was M. le Cardinal Fesch. One day, when the Emperor had come to visit his uncle, the worthy Curé, who was waiting in the anteroom, found himself present when His Majesty passed. Napoleon, on finding himself observed with a certain curiosity by this old man, turned round and said abruptly:—\n\n“Who is this good man who is staring at me?”\n\n“Sire,” said M. Myriel, “you are look

Traceback (most recent call last):
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\gradio\queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\gradio\route_utils.py", line 288, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\gradio\blocks.py", line 1931, in process_api
    result = await self.call_function(
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\gradio\blocks.py", line 1516, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\anyio\_backends\_asyncio.py", line 2177, in ru

Keyboard interruption in main thread... closing server.


In [3]:
import gradio as gr
import shutil

def save_file(file):
    if file is not None:
        # file 은 임시 저장 결로를 나타내는 듯
        shutil.copy(file, "save_file.txt")

with gr.Blocks() as app:
    file_input = gr.File(label="업로드 파일")
    send_btn = gr.Button("파일 저장하기")

    send_btn.click(fn=save_file, inputs=file_input, outputs=None)

app.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [ ]:
import gradio as gr
import os

def save_uploaded_file(file, directory):
    # 사용자가 지정한 디렉토리가 존재하지 않으면 생성합니다.
    if not os.path.exists(directory):
        os.makedirs(directory)

    # 파일 이름을 가져옵니다.
    file_name = file.name
    # 파일 내용을 읽어옵니다.
    file_content = file.read()

    # 파일을 지정된 경로에 저장합니다.
    save_path = os.path.join(directory, file_name)
    with open(save_path, "wb") as f:
        f.write(file_content)
    
    return f"파일이 저장되었습니다: {save_path}"

# Gradio 인터페이스를 설정합니다.
iface = gr.Interface(
    fn=save_uploaded_file,
    inputs=[gr.File(type="file"), gr.Textbox(label="저장할 디렉토리 경로를 입력하세요")],
    outputs="text",
    title="파일 업로드 및 저장"
)

# 인터페이스를 실행합니다.
iface.launch()
